In [1]:
# Notebook 09: Robust Portfolio Construction & Backtesting

### 🎯 Goal:
#Construct regime-aware robust portfolios using the best regime detection method and compare their performance with the baseline entropy-volatility regime portfolios.

### ✅ Step 1: Setup & Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

# Metrics
from scipy.stats import skew, kurtosis

# Optimization
from scipy.optimize import minimize

In [9]:
### ✅ Step 2: Load Data
# Load processed dataset with final regime column from previous notebook
# Choose the best regime column (e.g., 'gmm_regime' or 'hmm_regime')
data = pd.read_csv("../data/full_regimes_combined.csv", parse_dates=['Date'])
data.head()

,Date,vol_regime,cluster_regime,gmm_regime,hmm_regime
0,2015-04-21,2,1,1,0
1,2015-04-22,2,1,1,0
2,2015-04-23,2,1,1,0
3,2015-04-27,2,1,1,0
4,2015-04-28,2,1,1,0


In [10]:
### ✅ Step 3: Portfolio Construction Function (per regime)

def optimize_portfolio(returns, risk_aversion=1.0):
    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    def objective(weights):
        port_return = np.dot(weights, mean_returns)
        port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return - (port_return - risk_aversion * port_vol)

    n_assets = len(mean_returns)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for _ in range(n_assets))
    initial_weights = np.ones(n_assets) / n_assets

    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

In [11]:
### ✅ Step 4: Regime-wise Portfolio Construction

def construct_regime_portfolios(data, returns_columns, regime_column):
    regimes = data[regime_column].unique()
    portfolios = {}

    for r in regimes:
        subdata = data[data[regime_column] == r][returns_columns]
        weights = optimize_portfolio(subdata)
        portfolios[r] = weights

    return portfolios

In [15]:
### ✅ Step 5: Apply Portfolio Over Time

def construct_regime_portfolios(data, returns_columns, regime_column):
    regimes = data[regime_column].unique()
    portfolios = {}

    for r in regimes:
        subdata = data[data[regime_column] == r][returns_columns]

        # Skip if subdata is empty or has NaNs only
        if subdata.empty or subdata.dropna(how='all').empty:
            print(f"Skipping regime {r} due to lack of data.")
            continue

        weights = optimize_portfolio(subdata)
        portfolios[r] = weights

    return portfolios


In [16]:
### ✅ Step 6: Evaluate Performance

def evaluate_strategy(returns):
    cumulative = (1 + returns).cumprod()
    annual_return = returns.mean() * 252
    annual_vol = returns.std() * np.sqrt(252)
    sharpe = annual_return / annual_vol
    max_drawdown = (cumulative / cumulative.cummax() - 1).min()

    return {
        "Annual Return": annual_return,
        "Annual Volatility": annual_vol,
        "Sharpe Ratio": sharpe,
        "Max Drawdown": max_drawdown
    }

In [17]:
### ✅ Step 7: Run For Final Regimes

# Define returns columns (e.g., sectoral index returns)
returns_columns = [col for col in data.columns if '_ret' in col]

# Choose regime column to test (e.g., 'gmm_regime')
final_regime_column = 'gmm_regime'

# Step A: Create regime-aware portfolio
portfolios = construct_regime_portfolios(data, returns_columns, final_regime_column)

# Step B: Apply weights over time
strategy_returns = compute_portfolio_returns(data, portfolios, returns_columns, final_regime_column)

# Step C: Evaluate performance
metrics = evaluate_strategy(strategy_returns)
print(metrics)

Skipping regime 1 due to lack of data.
Skipping regime 2 due to lack of data.
Skipping regime 0 due to lack of data.
Skipping regime 3 due to lack of data.
{'Annual Return': np.float64(0.0), 'Annual Volatility': np.float64(0.0), 'Sharpe Ratio': np.float64(nan), 'Max Drawdown': np.float64(0.0)}


C:\Users\iamta\AppData\Local\Temp\ipykernel_10772\3360976806.py:7: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = annual_return / annual_vol
